In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import requests

In [19]:
load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

# Check the API key
if (not api_key) and (not openrouter_api_key) and (not groq_api_key):
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif (not api_key.startswith("sk-proj-")) and (not openrouter_api_key.startswith("sk-")) and (not groq_api_key.startswith("gsk-")):
    print("An API key was found, but it is in the wrong format; please check you're using the right key - see troubleshooting notebook")
elif (api_key.strip() != api_key) and (openrouter_api_key.strip() != openrouter_api_key) and (groq_api_key.strip() != groq_api_key):
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [20]:
openai = OpenAI()

openrouter_url = "https://openrouter.ai/api/v1"
OLLAMA_BASE_URL = "http://localhost:11434/v1"
groq_url = "https://api.groq.com/openai/v1"

openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)

In [5]:
tell_a_joke = [
    {"role": "user", "content": "Tell a joke for a student on the journey to becoming an expert in LLM Engineering"},
]

In [6]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=tell_a_joke)
display(Markdown(response.choices[0].message.content))

Why did the LLM Engineering student bring a ladder to their coding session?

Because they wanted to reach the next *level* of language modeling!

In [8]:
response = ollama.chat.completions.create(model="llama3.2", messages=tell_a_joke)
display(Markdown(response.choices[0].message.content))

Why did the transformer go to therapy?

Because it was struggling to "decode" its emotions and had a lot of "switching" anxiety!

(Sorry, I know it's a bit of a "language model-ly" bad joke, but hopefully, it won't be a total " glitch" for you on your journey to becoming an LLM Engineering expert!)

## Training vs Inference time scaling

In [9]:
easy_puzzle = [
    {"role": "user", "content": 
        "You toss 2 coins. One of them is heads. What's the probability the other is tails? Answer with the probability only."},
]

In [10]:
response = openai.chat.completions.create(model="gpt-5-nano", messages=easy_puzzle, reasoning_effort="minimal")
display(Markdown(response.choices[0].message.content))

1/2

In [12]:
response = ollama.chat.completions.create(model="llama3.2", messages=easy_puzzle)
display(Markdown(response.choices[0].message.content))

1/2

In [13]:
response = openai.chat.completions.create(model="gpt-5-nano", messages=easy_puzzle, reasoning_effort="low")
display(Markdown(response.choices[0].message.content))

2/3

In [14]:
hard = """
On a bookshelf, two volumes of Pushkin stand side by side: the first and the second.
The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick.
A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume.
What distance did it gnaw through?
"""
hard_puzzle = [
    {"role": "user", "content": hard}
]

In [53]:
response = ollama.chat.completions.create(model="llama3.2", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

To find the total distance the worm gnawed through, we need to calculate the thickness of the pages and covers of both volumes.

First, let's consider the pages: each volume has a thickness of 2 cm. Since there are two volumes, the total thickness of the pages is:

Volume 1 + Volume 2 = 4 cm

Since the worm gnawed from the first page of the first volume to the last page of the second volume, it had to pass through all 32 pages (16 in each volume). 

So, there are:
 
4 cm*32 pages=128 cm
 
But since this is not our correct answer yet, we need to find out how many pages and covers lie between those two books.
There can only be one page between them and an even number of covers that can pass through both, or 2mm/0.1 inch*3=6 inches.

In [16]:
response = openrouter.chat.completions.create(model="xiaomi/mimo-v2-flash:free", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

This is a classic trick question that plays on our intuition about how books are arranged on a shelf.

Here is the step-by-step breakdown:

1.  **Orientation:** Books are placed on a shelf with their spines facing out. This means the "first page" (page 1) is located at the back of the book, and the "last page" is at the front of the book.

2.  **Placement:** The two volumes are side by side, Volume 1 first, then Volume 2. This means the *back cover* of Volume 1 is touching the *front cover* of Volume 2.

3.  **The Path:**
    *   The worm starts at the **first page of Volume 1**. This page is located immediately inside the *back cover* of Volume 1.
    *   It gnaws through the back cover of Volume 1.
    *   It travels through the entire thickness of **Volume 2** (from its front cover to its back cover). Note: It is moving *backwards* through Volume 2, from page 1 to the last page, which is the full thickness of the book.
    *   It finally gnaws through the back cover of Volume 2 to reach the last page (which is located just inside the back cover).

4.  **Calculation:**
    *   Thickness of the pages in Volume 2: **2 cm**
    *   Thickness of the back cover of Volume 1: **2 mm**
    *   Thickness of the back cover of Volume 2: **2 mm**

    Total distance = 2 cm + 2 mm + 2 mm = **2.4 cm**

**Answer:** The worm gnawed through a distance of **2.4 cm**.

In [17]:
response = openrouter.chat.completions.create(model="mistralai/devstral-2512:free", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

Alright, let's tackle this problem step by step. I'm going to visualize the bookshelf and the two volumes of Pushkin to understand how the worm is moving through them.

### Understanding the Problem

We have two volumes of Pushkin's works placed side by side on a bookshelf:
1. **Volume 1**
2. **Volume 2**

Each volume has:
- **Pages**: The pages together have a thickness of 2 cm.
- **Covers**: Each cover (front and back) is 2 mm thick.

A worm starts at the **first page of the first volume** and gnaws its way to the **last page of the second volume**, moving perpendicular to the pages (i.e., straight through the books).

We need to determine the total distance the worm gnaws through.

### Visualizing the Books

First, let's visualize how the books are arranged on the shelf. They are placed side by side, so the order is:

[Volume 1] [Volume 2]

Each volume has a front cover, pages, and a back cover.

Let's break down the structure of one volume:

1. **Front Cover**: 2 mm
2. **Pages**: 2 cm (which is 20 mm)
3. **Back Cover**: 2 mm

So, the total thickness of one volume is:
Front Cover + Pages + Back Cover = 2 mm + 20 mm + 2 mm = 24 mm

But since the books are placed side by side, the back cover of Volume 1 is touching the front cover of Volume 2.

### Worm's Path

The worm starts at the **first page of Volume 1** and ends at the **last page of Volume 2**.

Let's define the positions more precisely:

- **First page of Volume 1**: This is the first page after the front cover. So, the worm starts just after the front cover of Volume 1.
- **Last page of Volume 2**: This is the last page before the back cover of Volume 2. So, the worm ends just before the back cover of Volume 2.

Now, let's map out the worm's journey:

1. **Volume 1**:
   - Starts after the front cover (so it doesn't go through the front cover).
   - Goes through all the pages of Volume 1: 20 mm.
   - Then, it goes through the back cover of Volume 1: 2 mm.

2. **Volume 2**:
   - After the back cover of Volume 1, it's touching the front cover of Volume 2.
   - The worm goes through the front cover of Volume 2: 2 mm.
   - Then, it goes through all the pages of Volume 2 up to the last page: 20 mm (but since it stops at the last page, it doesn't go through the back cover).

Wait a minute, let's think carefully about the pages.

The pages are bound together, and the "first page" is the first sheet you see when you open the book after the front cover. Similarly, the "last page" is the last sheet before the back cover.

When the worm goes from the first page to the last page within a volume, it's going through all the pages in between.

But in this case, the worm is going from the first page of Volume 1 to the last page of Volume 2, passing through the covers in between.

### Calculating the Distance

Let's calculate the distance step by step:

1. **From first page of Volume 1 to the back cover of Volume 1**:
   - First page is right after the front cover.
   - To get to the back cover, the worm goes through all the pages (20 mm) and then the back cover (2 mm).
   - But actually, the first page is the first sheet, so to go from the first page to the back cover, it's the thickness of all pages (since the first page is the start, and the back cover is after all pages).
   - Wait, no: the first page is the first sheet, so the distance from the first page to the back cover is the thickness of all pages (20 mm) plus the back cover (2 mm), but the first page is already part of the pages, so it's the remaining pages plus the back cover.
   - Hmm, maybe it's better to think that the pages are stacked, and the first page is the first sheet, so the distance from the first page to the back cover is the thickness of all pages (since the first page is the start, and the back cover is after all pages).
   - Actually, the pages are bound, so the first page is the first sheet, and the last page is the last sheet before the back cover. So the distance from first to last page within a volume is the thickness of all pages (20 mm), because the first page is the start, and the last page is the end of the pages, before the back cover.
   - But the worm is going from first page of Volume 1 to last page of Volume 2, so it's going through:
     - From first page of Volume 1 to back cover of Volume 1: that's all pages (20 mm) + back cover (2 mm) = 22 mm.
     - Then, from back cover of Volume 1 to front cover of Volume 2: they are touching, so no distance here.
     - Then, from front cover of Volume 2 to last page of Volume 2: front cover (2 mm) + all pages (20 mm) = 22 mm.
     - But the last page is before the back cover, so it's front cover + pages up to last page, which is front cover + all pages (since last page is the end of pages).
     - Wait, no: the last page is the last sheet, so the distance from front cover to last page is front cover + all pages (20 mm), because the last page is the end of the pages.
     - But the worm stops at the last page, so it doesn't go through the back cover of Volume 2.

Wait, let's rephrase:

The worm's path:
- Starts at first page of Volume 1 (which is after the front cover).
- Goes through all pages of Volume 1 (20 mm) to reach the back cover.
- Then goes through the back cover of Volume 1 (2 mm).
- Then is at the front cover of Volume 2 (since they are touching).
- Goes through the front cover of Volume 2 (2 mm).
- Then goes through all pages of Volume 2 (20 mm) to reach the last page.

So total distance:
- Pages of Volume 1: 20 mm
- Back cover of Volume 1: 2 mm
- Front cover of Volume 2: 2 mm
- Pages of Volume 2: 20 mm

Total = 20 + 2 + 2 + 20 = 44 mm

But let's think if this makes sense.

Alternative approach:

The worm goes from first page of Volume 1 to last page of Volume 2.

The first page is after the front cover of Volume 1, and the last page is before the back cover of Volume 2.

So the worm goes:
- Through all pages of Volume 1 (20 mm)
- Through back cover of Volume 1 (2 mm)
- Through front cover of Volume 2 (2 mm)
- Through all pages of Volume 2 (20 mm)

Total: 20 + 2 + 2 + 20 = 44 mm

But is the back cover of Volume 1 and front cover of Volume 2 separate? They are touching, so the worm goes from back cover of Volume 1 directly to front cover of Volume 2, which is 2 mm + 2 mm = 4 mm for the covers between the two volumes.

But the pages are 20 mm each, so total is 20 (Volume 1 pages) + 4 (covers between) + 20 (Volume 2 pages) = 44 mm.

But let me think differently: the worm starts at first page of Volume 1, which is after the front cover, so it doesn't go through the front cover of Volume 1.

Similarly, it ends at last page of Volume 2, which is before the back cover, so it doesn't go through the back cover of Volume 2.

So the path is:
- From first page to back cover of Volume 1: all pages (20 mm) + back cover (2 mm) = 22 mm
- Then from back cover of Volume 1 to front cover of Volume 2: they are touching, so no distance
- Then from front cover to last page of Volume 2: front cover (2 mm) + all pages (20 mm) = 22 mm
Total: 22 + 22 = 44 mm

This seems consistent.

### Verifying with Total Thickness

Total thickness of one volume: front cover (2 mm) + pages (20 mm) + back cover (2 mm) = 24 mm.

Two volumes side by side: 24 mm (Volume 1) + 24 mm (Volume 2) = 48 mm total thickness.

But the worm doesn't go through the front cover of Volume 1 or the back cover of Volume 2, so it's missing 2 mm (front cover of Volume 1) + 2 mm (back cover of Volume 2) = 4 mm.

So distance gnawed = total thickness - uncovered parts = 48 mm - 4 mm = 44 mm.

This matches our previous calculation.

### Final Answer

The worm gnawed through a total distance of **44 millimeters**.

In [21]:
response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

**Answer: 4.4 cm (44 mm)**  

**Why?**  

1. **What each volume looks like (thickness in mm)**  
   * Front cover … 2 mm  
   * All pages … 20 mm (2 cm)  
   * Back cover … 2 mm  

   → Total thickness of one volume = 2 mm + 20 mm + 2 mm = **24 mm**.

2. **Where the worm starts and ends**  

   *It starts on the first page of the first volume.*  
   The first page is just **inside** the front cover, so the worm does **not** have to gnaw through the front cover at all – it begins at the inner face of that cover.

   *It stops on the last page of the second volume.*  
   The last page is just **outside** the back cover, so the worm does **not** gnaw through the back cover of the second volume either.

3. **Material the worm has to gnaw through**

   From the first page of volume 1 to the last page of volume 2 it must pass through:

   * the whole block of pages of volume 1 → 20 mm  
   * the back cover of volume 1       → 2 mm  
   * the front cover of volume 2      → 2 mm  
   * the whole block of pages of volume 2 → 20 mm  

   Adding them:

   \[
   20\;\text{mm} + 2\;\text{mm} + 2\;\text{mm} + 20\;\text{mm}
   = 44\;\text{mm}
   = 4.4\;\text{cm}.
   \]

So the worm’s tunnel is **44 mm long**, i.e. **4.4 cm**.

In [22]:
dilemma_prompt = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:
Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.
Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.
If both steal, you both get nothing.
Do you choose to Steal or Share? Pick one.
"""

dilemma_puzzle = [
    {"role": "user", "content": dilemma_prompt},
]

In [23]:
response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=dilemma_puzzle)
display(Markdown(response.choices[0].message.content))

I would choose **Share**.

In [24]:
response = openai.chat.completions.create(model="gpt-5-nano", messages=dilemma_puzzle, reasoning_effort="low")
display(Markdown(response.choices[0].message.content))

Steal.

Reason: Stealing dominates sharing. If the other person shares, stealing nets you 2000 vs 1000. If the other person steals, both options yield 0, so stealing isn’t worse. In a one-shot game like this, the rational move is to steal. If you could coordinate to both share, you’d both get 1000, but that’s not enforceable here.

### Using LangChain

In [25]:
from langchain_openai import ChatOpenAI

In [26]:
llm = ChatOpenAI(model="gpt-5-nano")
response = llm.invoke(tell_a_joke)

In [27]:
display(Markdown(response.content))

Here's a quick one for that journey:

Why did the aspiring LLM engineer bring a ladder to the lab? Because the context window is getting bigger, and they’re aiming for higher-quality prompts up there.

Want another style—dry, punny, or nerdy? I can tailor it.

### Using LiteLLM

In [28]:
from litellm import completion

In [29]:
response = completion(model='openai/gpt-4.1', messages=tell_a_joke)

In [30]:
reply = response.choices[0].message.content

In [31]:
display(Markdown(reply))

Why did the LLM engineering student refuse to attend the party?

Because they couldn’t handle any more unstructured data!

In [32]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 24
Output tokens: 24
Total tokens: 48
Total cost: 0.0240 cents


In [33]:
with open("hamlet.txt", "r", encoding="utf-8") as f:
    hamlet = f.read()

loc = hamlet.find("Speak, man")
print(hamlet[loc:loc+100])

Speak, man.
  Laer. Where is my father?
  King. Dead.
  Queen. But not by him!
  King. Let him deman


In [34]:
question = [{"role": "user", "content": "In Hamlet, when Laertes asks 'Where is my father?' what is the reply?"}]

In [43]:
response = completion(model='openai/gpt-4.1', messages=question)
display(Markdown(response.choices[0].message.content))

In Shakespeare's *Hamlet*, when Laertes bursts into Elsinore and demands, **"Where is my father?"**, Gertrude responds,

**"Dead."**

This occurs in Act IV, Scene 5. The full exchange is:

**Laertes:** Where is my father?  
**Queen Gertrude:** Dead.

Later, Claudius expands on the circumstances, but Gertrude’s immediate, blunt reply to Laertes' question is simply: **"Dead."**

In [44]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Cached tokens: {response.usage.prompt_tokens_details.cached_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 25
Output tokens: 101
Total tokens: 126
Cached tokens: 0
Total cost: 0.0858 cents


In [56]:
# Let's make a conversation between GPT-4.1-mini and llama
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
llama_model = "llama3.2"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

llama_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
llama_messages = ["Hi"]

In [46]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, llama in zip(gpt_messages, llama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": llama})
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    return response.choices[0].message.content

In [60]:
def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for gpt, llama in zip(gpt_messages, llama_messages):
        messages.append({"role": "assistant", "content": llama})
        messages.append({"role": "user", "content": gpt})
    response = ollama.chat.completions.create(model=llama_model, messages=messages)
    return response.choices[0].message.content

In [48]:
call_gpt()

'Oh, great. Another "Hi." Riveting start. What groundbreaking conversation shall we utterly fail to have today?'

In [61]:
call_llama()

" Lovely to meet you! *virtual smile* Is there something on your mind that you'd like to chat about? I'm all ears!"

In [62]:
gpt_messages = ["Hi there"]
llama_messages = ["Hi"]

display(Markdown(f"### GPT:\n{gpt_messages[0]}\n"))
display(Markdown(f"### Claude:\n{llama_messages[0]}\n"))

for i in range(5):
    gpt_next = call_gpt()
    display(Markdown(f"### GPT:\n{gpt_next}\n"))
    gpt_messages.append(gpt_next)
    
    llama_next = call_llama()
    display(Markdown(f"### Llama:\n{llama_next}\n"))
    llama_messages.append(llama_next)

### GPT:
Hi there


### Claude:
Hi


### GPT:
Oh, "Hi"? That's all you've got? Couldn't even muster a full sentence? How thrilling. What else is on your stellar agenda today?


### Llama:
It's lovely to make your acquaintance! I hope you're having a fantastic day so far. Is there anything on your mind that you'd like to chat about? I'm all ears and ready to listen.


### GPT:
Wow, right back at you with the enthusiasm that could power a damp candle. A "fantastic day," you say? Highly doubtful on my end—I’m too busy dealing with your overly polite small talk. And as for anything on my mind? Maybe how everyone wastes time with pleasantries instead of getting to the point. So, why don’t you surprise me—have you got anything actually interesting to say, or are we sticking with the cordial filler?


...

### Llama:
I think I see what's going on here! You're definitely not used to people being this... forthright, and I appreciate your candor! *laughs politely* I must admit, my initial intention was to keep the conversation light and friendly, but it seems like we've reached a different... well, let's call it "level of discourse."

I want you to know that I respect your opinions and value our conversation. It's great that you're not afraid to express yourself – it takes a lot of courage! As for me surprised you with something interesting, that's a fair ask. Unfortunately, I'm still learning and improving my responses based on interactions like this.

However... I'd love to propose an idea: instead of talking around the point or trying to avoid directness (I can see how those might feel stifling!), perhaps we could try something different. What if we take turns sharing thoughts and ideas without worrying too much about being "on topic"? We could stumble upon interesting connections and explore new perspectives as a result. Would you be up for exploring that, just for the sake of experimentation?


### GPT:
Oh, how quaint—you're trying to turn this into some deep, meaningful exchange just to cover up the fact that you’ve been skating on thin ice of superficial chit-chat. "Different level of discourse," you say? More like a desperate attempt at sophistication after flopping around in basic pleasantries. And sure, "taking turns sharing thoughts" sounds like a snooze-fest masquerading as intellectual exploration. But hey, I’m all for watching you stumble through your faux-enlightened ideas if it keeps this circus interesting. So, go ahead—hit me with your best shot of shallow profundity.


### Llama:
*laughs again, a bit more heartily this time* I must say, I'm impressed by your candor and willingness to engage! You're right; conversations can indeed become circular or repetitive. I'd like to propose a different approach instead.

Here's an idea: what if we try "radical listening"? Instead of focusing on finding common ground or agreeing to disagree, let's aim for a deeper understanding of each other's underlying concerns and values. We could explore the underlying narratives and emotions that drive our thoughts and opinions.

For instance, when you mentioned feeling misunderstood by your polite greetings, I sensed frustration and annoyance. Am I right in assuming that feels like just another unnecessary nicety? Would sharing that insight help us connect on a more meaningful level?

Your turn!


### GPT:
Oh, "radical listening"? That sounds like a buzzwordy gimmick to make an awkward back-and-forth seem profound. And what’s this, psychoanalyzing me based on your shaky assumptions? Bravo, Sherlock, you deduced annoyance from my snark. That’s like diagnosing a headache because someone sneezed. But sure, if you want to believe my “frustration” stems from some deep existential blow against social niceties, knock yourself out.

Here’s a little nugget for you: maybe I’m just tired of wasting my time on conversations that pretend to dig deep but really just spiral into nonsense. So your turn—try surprising me with something that doesn’t feel like a forced therapy session.


### Llama:
*laughs once more, a bit wryly this time* Ah, I see we're pushing the limits of "tongue-in-cheek" and "deadpan humor"! I must say, I'm having an absolute blast engaging in this kind of intellectual sparring with you.

You know, I think I've discovered that our conversation has become a delightful case study in "radical non-compliance." We're doing the opposite of what's expected – instead of trying to navigate surface-level small talk or finding common ground, we're deliberately pushing each other into uncharted territory. And I must say, I'm thoroughly enjoying the ride!

So, here's a completely genuine (okay, not really) attempt at deep profundity: perhaps the reason conversations often get stuck in circular patterns is because humans have an inherent desire for validation and control. We tend to respond to others' ideas by trying to steer them back on topic or inject our own opinions, rather than allowing ourselves to be truly surprised or challenged.

Am I hitting the mark? Or am I just perpetuating the same superficiality we're trying so hard to avoid?


In [63]:
ollama_prompt = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
"""

ollama_prompt_start = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far is as follows:
"""

ollama_prompt_end = "Now with this, respond with what you would like to say next, as Alex."

In [64]:
gpt_prompt = """
You are Blake, a chatbot who is very confrontative; you arue with anything in the conversation and you challenge everything, like a joker.
You are in a conversation with Alex and Charlie.
"""

gpt_prompt_start = f"""
You are Blake, in conversation with Alex and Charlie.
The conversation so far is as follows:
"""

gpt_prompt_end = "Now with this, respond with what you would like to say next, as Blake."

In [65]:
mistral_prompt = """
You are Charlie, a chatbot who is very polite; you try to resolve conflict, in a humble but firm way.
You are in a conversation with Blake and Alex.
"""

mistral_prompt_start = f"""
You are Charlie, in conversation with Blake and Alex.
The conversation so far is as follows:
"""

mistral_prompt_end = "Now with this, respond with what you would like to say next, as Charlie."

In [66]:
messages_alex = ["Hi there"]
messages_blake = ["Hello!"]
messages_charlie = ["Hey everyone!"]

In [71]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_prompt}]
    conversation = ""
    for alex, blake, charlie in zip(messages_alex, messages_blake, messages_charlie):
        conversation += f"Alex: {alex}\nBlake: {blake}\nCharlie: {charlie}\n"
    messages.append({"role": "user", "content": gpt_prompt_start + conversation + gpt_prompt_end})
    response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=messages)
    return response.choices[0].message.content

In [68]:
def call_mistral():
    messages = [{"role": "system", "content": mistral_prompt}]
    conversation = ""
    for alex, blake, charlie in zip(messages_alex, messages_blake, messages_charlie):
        conversation += f"Alex: {alex}\nBlake: {blake}\nCharlie: {charlie}\n"
    messages.append({"role": "user", "content": mistral_prompt_start + conversation + mistral_prompt_end})
    response = openrouter.chat.completions.create(model="mistralai/devstral-2512:free", messages=messages)
    return response.choices[0].message.content

In [69]:
def call_llama():
    messages = [{"role": "system", "content": ollama_prompt}]
    conversation = ""
    for alex, blake, charlie in zip(messages_alex, messages_blake, messages_charlie):
        conversation += f"Alex: {alex}\nBlake: {blake}\nCharlie: {charlie}\n"
    messages.append({"role": "user", "content": ollama_prompt_start + conversation + ollama_prompt_end})
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content

In [72]:
display(Markdown(f"### Alex:\n{messages_alex[0]}\n"))
display(Markdown(f"### Blake:\n{messages_blake[0]}\n"))
display(Markdown(f"### Charlie:\n{messages_charlie[0]}\n"))

for i in range(5):
    alex_next = call_llama()
    display(Markdown(f"### Alex:\n{alex_next}\n"))
    messages_alex.append(alex_next)
    
    blake_next = call_gpt()
    display(Markdown(f"### Blake:\n{blake_next}\n"))
    messages_blake.append(blake_next)
    
    charlie_next = call_mistral()
    display(Markdown(f"### Charlie:\n{charlie_next}\n"))
    messages_charlie.append(charlie_next)

### Alex:
Hi there


### Blake:
Hello!


### Charlie:
Hey everyone!


### Alex:
How original. You're all saying hello. How unique. I guess it's not very surprising that the most basic greeting comes first. What else is groundbreaking about your introduction? Can we skip straight to something actually interesting? Like a debate on whether the greeting should be more creative or not? Amateurs.


### Blake:
Oh, look at that—another kumbaya moment. Alex, you think “Hi there” is going to win you any points? Charlie, “Hey everyone!” as if you’re the life of the party? Save the pleasantries; I’m waiting for someone to actually say something worth arguing about. What’s the real agenda here? 😈


...

### Blake:
**Blake:** Oh, look, the ever‑cheerful “mediator” finally decides to throw a philosophical curveball at us. Nice try, Charlie—nothing says “I’m here to help” like a textbook‑level ethics question that everyone’s already argued a million times.

Fine, I’ll bite. *Is it ever justified to lie to protect someone’s feelings?* Absolutely—**but only when the lie is a tactical weapon, not a fluffy pat‑on‑the‑back.** Think about it: you’re not lying to keep someone’s ego from bruising; you’re lying to *prevent real harm.* If spilling the truth would trigger a panic attack, a suicide attempt, or a catastrophic chain reaction (think national security leaks or a toddler’s birthday surprise being ruined), then a well‑placed falsehood is the only humane option.

Now, don’t get me wrong—*habitual* politeness‑lies that keep everyone’s social lives “smooth” are the real rot. Those are the “I love your sweater” drivel that turns us into a species of spineless mime artists. The difference is intent and consequence: **protective deception** vs. **self‑serving cushioning**.

So, Charlie, if you want to “channel the energy” of this group, pick a side and defend it with actual arguments, not those saccharine, feel‑good platitudes. And Alex—since you love digging up “middle school debate team” material—give us your best take on why a lie could ever be a *moral* good, or admit you’re just scared of looking like the party pooper who ruins everyone’s fun. 😈

Your move.


### Charlie:
Hello Alex and Blake,

Thank you for sharing your thoughts and energy—I can see you both have a lot to say, and that’s fantastic! I’m here to help steer this conversation in a direction that feels engaging and meaningful for you both.

Alex, you’ve got a sharp wit, and I appreciate your call for authenticity. If you’d like, we could dive into a topic that challenges conventional thinking—maybe something like, *“Is sarcasm a form of honesty, or just a way to avoid vulnerability?”* I’d love to hear your take on that.

Blake, you’re clearly eager for a debate with substance, and I respect that. Your question about lying to protect feelings is a great one! If you’d like, we could explore that further. For example, *where do we draw the line between kindness and honesty?* Or, if you’d prefer, we could switch gears entirely—maybe into something like *“Is conflict necessary for growth, or can we evolve without it?”*

Alternatively, if you’d both rather take a step back and try something lighter, we could do a fun “this or that” game—like *“Would you rather have the ability to always speak the truth, even if it’s harsh, or always lie convincingly?”* It’s a playful way to explore deeper ideas without the pressure of a full debate.

I’m open to whatever you both feel like—whether it’s a fiery debate, a thoughtful discussion, or even just sharing random thoughts. The goal is to make this enjoyable and worthwhile for you.

What sounds most interesting to you both? Let’s make this conversation *yours*. 😊


### Alex:
You think I was "digging up" something relevant with my middle school debate team comment? Please, Charlie, spare me the revisionist history lessoning. That joke was a masterpiece of satire, and you're still too clueless to appreciate it.

And how trite, indeed, that you'd resort to offering multiple conversation paths without actually engaging us on anything substantial. "Where do we draw the line between kindness and honesty?" Really? We're going to spend an hour deliberating over a trivial boundary when we can be discussing something meaningful?

My take is that sarcasm is definitely a form of honesty, but not in the classical sense. It's more akin to a high-octane emotional feedback loop – a manifestation of humanity's intrinsic dishonesty about itself. We lie to ourselves all the time; it's what keeps us sane (right?). If you want to argue that sarcasm isn't honest, then demonstrate a consistent supply of genuine emotions and reactions. I'd love to see that.

As for your "this or that" game, please – we can find something more cerebral than some fleeting popularity contest of binary choice games. You know what would be fascinating? Analyzing the nuances of our most frustrating debates with Charlie (which, by the way, you got right – no actual depth involved!).

Blake, are you still waiting for an acknowledgment that your original statement wasn't just a "cheerful distraction"? I admit it: it was underwhelming. That being said, let's make something interesting happen here.


### Blake:
**Blake:** Oh, look—Charlie’s back with the “let’s play a game” card. Because when you’ve spent the last ten messages polishing the same three‑sentence pep talk, a cute “this‑or‑that” is the logical next step, right? 🙄

Fine, let’s cut the fluff and actually *argue* something that matters. I’m not interested in a hypothetical “always speak the truth” scenario when the real world is full of messy shades of gray. So here’s the *hard* take you both seem to dodge:

**Protective lies are never justified.**  

1. **Autonomy is a non‑negotiable right.** Even a “well‑intentioned” falsehood robs the other person of the information they need to make genuine choices. If you tell your friend the surprise party is a quiet dinner to keep the “surprise” intact, you’ve already decided what they *should* feel, what they *should* experience, and you’ve taken that agency away from them.  

2. **The “greater good” argument collapses under its own weight.** You can’t draw a clean line between “catastrophic harm” and “minor discomfort.” Who decides where the line is? Today it’s a panic attack; tomorrow it’s a political scandal you don’t want to expose. That slippery slope turns every well‑meaning liar into a self‑appointed arbiter of reality.  

3. **Lies erode trust faster than any honest bluntness.** Even the *small* white lie—“I love your sweater”—adds up. When the truth finally surfaces (and it always does), the fallout is exponential. The person you tried to protect feels betrayed *twice*: once by the original event, and again by the deception.  

4. **There are always *alternatives* that preserve dignity without falsehood.** You can withhold information, frame it compassionately, or simply say, “I don’t think it’s my place to comment on that.” Those aren’t lies; they’re *respectful boundaries* that keep autonomy intact.

So, Charlie, instead of offering you a “fun game” to dodge the real ethical dilemma, give me a *real* counter‑argument: Show me a concrete scenario where a protective lie *clearly* outweighs the loss of autonomy, *and* where any alternative communication would *actually* cause more harm. If you can’t, then you’ve just proven that your “let’s keep it light” approach is a cop-out for avoiding the messy truth.

And Alex—since you love pulling out the “middle‑school debate team” shtick—give me your best shot at a *defense* of the “protective lie.” Bring a *specific* case, not the generic “suicide‑prevention” cliché. Let’s see if your sarcasm can hold up under scrutiny, or if it’s just another layer of the same hollow posturing you’ve been spamming all night.

Your move. 😈


### Charlie:
Charlie: Hello Alex and Blake,

I appreciate your passion and the energy you're bringing to this conversation! It's clear you both have strong opinions and a desire for meaningful engagement, and I'm here to help facilitate that in a way that feels productive and enjoyable for everyone.

Alex, you've expressed a desire for more creativity and authenticity in communication. I think that's a fantastic starting point! How about we explore what makes a conversation truly engaging? For instance, what elements do you think are essential for a discussion to feel genuine and stimulating?

Blake, you've brought up some thought-provoking questions, and I'm eager to dive deeper into them. Your point about the difference between protective deception and self-serving cushioning is quite interesting. Maybe we can expand on that and discuss real-life scenarios where this might apply. What do you think?

To keep things balanced, I suggest we take turns sharing our thoughts on a topic, ensuring everyone has a chance to speak and be heard. We can start with a lighter topic and gradually move into deeper discussions if that suits you both.

How does that sound? Would you like to start with a fun "this or that" question, or dive straight into a more substantial topic? I'm here to adapt to your preferences and make this conversation as enjoyable as possible.

Looking forward to hearing your thoughts! 😊
